In [ ]:
! pip install -q transformers torch --quiet
! pip install -q transformers sentencepiece --quiet

In [ ]:
!pip install -U transformers accelerate datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 34.3 MB/s  0:00:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [datasets]


In [ ]:
import transformers
transformers.__version__


'4.57.3'

In [ ]:
!pip install --upgrade pip setuptools wheel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
!pip install -q transformers datasets evaluate seqeval accelerate


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install -q evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import pipeline
import os
import numpy as np
import torch
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import evaluate
from datasets import Dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving gpt.xlsx to gpt.xlsx


In [ ]:
df=pd.read_excel('gpt.xlsx')

In [ ]:
df.head()

,id,message_text,category,route_to_team,priority_level
0,1,I ordered two phones by mistake. I only need o...,Order Issue,Order Support Team,Medium
1,2,I ordered black shoes but received pink ones. ...,Order Issue,Order Support Team,High
2,3,The speaker arrived broken right out of the bo...,Order Issue,Order Support Team,High
3,4,I ordered 3 t-shirts but only received 2. The ...,Order Issue,Order Support Team,Medium
4,5,You sent the wrong size. I ordered L but recei...,Order Issue,Order Support Team,Medium


full fine-tuned BERT training pipeline

Train/test split & label encoding

In [ ]:
df['message_text'] = df['message_text'].fillna("").astype(str)
df['category'] = df['category'].fillna("Unknown").astype(str)

In [ ]:
# df = df[["message_text", "ncategory"]].copy()
# len(df), df["ncategory"].nunique(), df["ncategory"].unique()

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
le = LabelEncoder()
le.fit(train_df["category"])
train_df["label"] = le.transform(train_df["category"])
test_df["label"]  = le.transform(test_df["category"])

In [ ]:
label_mapping = dict(zip(range(len(le.classes_)), le.classes_.tolist()))
print("Label mapping (int -> label):", label_mapping)

Label mapping (int -> label): {0: 'Delivery Delay', 1: 'Fraud/Security', 2: 'Order Issue', 3: 'Return Issue', 4: 'Technical Issue', 5: 'Unknown'}


Tokenizer & model selection

In [ ]:
MODEL_NAME = "distilbert-base-uncased"   # or "bert-base-uncased" / "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_fn(examples):
    return tokenizer(examples["message_text"], truncation=True, padding=False, max_length=256)


In [ ]:
train_ds = Dataset.from_pandas(train_df[["message_text","label"]])
test_ds  = Dataset.from_pandas(test_df[["message_text","label"]])


In [ ]:
train_ds = train_ds.map(tokenize_fn, batched=True)
test_ds  = test_ds.map(tokenize_fn, batched=True)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [ ]:
columns = ['input_ids', 'attention_mask', 'label']
train_ds = train_ds.remove_columns([col for col in train_ds.column_names if col not in columns])
test_ds  = test_ds.remove_columns([col for col in test_ds.column_names if col not in columns])
train_ds.set_format(type='torch', columns=columns)
test_ds.set_format(type='torch', columns=columns)

In [ ]:
#train_ds.train_ds.map(lambda x: tokenize_fn(x),batched=True)
#test_ds.test_ds.map(lambda x: tokenize_fn(x),batched=True)

In [ ]:
num_labels = len(le.classes_)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(output_dir="./bert_ticket_classifier",
                                  eval_strategy="epoch",
                                  save_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=32,
                                  num_train_epochs=3,
                                  weight_decay=0.01,
                                  logging_steps=50,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1",
                                  greater_is_better=True,
                                  fp16=torch.cuda.is_available()
                                  )

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Metrics Function for Trainer

In [ ]:
accuracy_mterics = evaluate.load("accuracy")
f1_metrics = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_mterics.compute(predictions=preds, references=labels)["accuracy"]
    f1 = f1_metrics.compute(predictions=preds, references=labels, average="weighted")["f1"]
    precision = precision_metric.compute(predictions=preds, references=labels, average="weighted")["precision"]
    recall = recall_metric.compute(predictions=preds, references=labels, average="weighted")["recall"]
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


/tmp/ipython-input-2584988314.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.746488,0.181818,0.111111,0.102273,0.181818
2,No log,1.719139,0.090909,0.020202,0.011364,0.090909
3,No log,1.697101,0.090909,0.025974,0.015152,0.090909


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator

TrainOutput(global_step=9, training_loss=1.681749661763509, metrics={'train_runtime': 78.6003, 'train_samples_per_second': 1.527, 'train_steps_per_second': 0.115, 'total_flos': 716202960192.0, 'train_loss': 1.681749661763509, 'epoch': 3.0})

In [ ]:
pred_output = trainer.predict(test_ds)
logits = pred_output.predictions
preds = np.argmax(logits, axis=-1)
labels = pred_output.label_ids

# Human-readable classification report
print(classification_report(labels, preds, zero_division=0))
# Confusion matrix (integers)
cm = confusion_matrix(labels, preds)
print("Confusion matrix:\n", cm)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.33      0.33      0.33         3
           1       0.00      0.00      0.00         3
           2       0.12      1.00      0.22         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         3

    accuracy                           0.18        11
   macro avg       0.09      0.27      0.11        11
weighted avg       0.10      0.18      0.11        11

Confusion matrix:
 [[1 0 2 0 0]
 [0 0 3 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [2 0 1 0 0]]


In [ ]:
model_dir = "./bert_ticket_classifier/best_model"
os.makedirs(model_dir, exist_ok=True)
trainer.save_model(model_dir)
tokenizer.save_pretrained(model_dir)
# Save label encoder classes for inference
import json
with open(os.path.join(model_dir, "label_mapping.json"), "w") as f:
    json.dump(label_mapping, f)
print("Saved model to", model_dir)

Saved model to ./bert_ticket_classifier/best_model


In [ ]:
inference_tokenizer = AutoTokenizer.from_pretrained(model_dir)
inference_model = AutoModelForSequenceClassification.from_pretrained(model_dir)
inference_pipe = pipeline(
    "text-classification",
    model=inference_model,
    tokenizer=inference_tokenizer,
    top_k=None,
    device=0 if torch.cuda.is_available() else -1
)

# Helper: convert raw pipeline output to label + confidence
def predict_batch(texts, topk=1):
    outs = inference_pipe(texts, truncation=True, max_length=256)
    results = []
    for out in outs:
        # out is list of dicts [{'label': '0', 'score': 0.92}, ...] or sometimes [{'label': 'LABEL_0',...}]
        # find highest score
        best = max(out, key=lambda x: x["score"])
        # convert label id to int (sometimes 'LABEL_0' or '0')
        raw_label = best["label"]
        try:
            int_label = int(raw_label.split("_")[-1])  # handles LABEL_0 or '0'
        except:
            try:
                int_label = int(raw_label)
            except:
                # if mapping uses class names directly, try to map
                # fallback: use label_mapping inverse
                int_label = None
        confidence = best["score"]
        results.append((int_label, confidence))
    return results

# Example usage:
sample_texts = ["The refund has not arrived after 12 days, please check.", "Wrong item delivered."]
preds = predict_batch(sample_texts)
print(preds)
# Convert int label to string label using label_mapping
pred_labels = [label_mapping[p[0]] if p[0] is not None and p[0] in label_mapping else "Unknown" for p in preds]
print(pred_labels)

Device set to use cpu


[(2, 0.18425147235393524), (2, 0.19037044048309326)]
['Order Issue', 'Order Issue']


| Metric    | BERT  | Zero-shot |
| --------- | ----  | --------- |
| Accuracy  | X-0.18| Y-0.64    |
| Precision | X-0.10| Y-0.80    |
| Recall    | X-0.18| Y-0.64    |


Initialize Zero-Shot Classifier
✔ This model understands category names directly
❌ No need to train
✔ Works instantly

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
labels = ["Order Issue","Delivery Delay","Return Issue","Technical Issue","Fraud/Security"]
def predict_category(text):
  if not text:
    return 'No Text'
  out = classifier(text, candidate_labels = labels)
  return out['labels'][0]

Zero-shot classifier only accepts strings.

In [ ]:
df["predicted_category"] = df["message_text"].fillna('').apply(predict_category)

In [ ]:
routing ={
    "Order Issue": "Order Support Team",
    "Delivery Delay": "Logistics Team",
    "Return Issue": "Returns Team",
    "Technical Issue": "Tech Support Team",
    "Fraud/Security": "Fraud Prevention Team"
}

In [ ]:
df['predicted_routing'] = df['predicted_category'].map(routing)

In [ ]:
df[['message_text','predicted_category','predicted_routing']].head()

,message_text,predicted_category,predicted_routing
0,I ordered two phones by mistake. I only need o...,Return Issue,Returns Team
1,I ordered black shoes but received pink ones. ...,Order Issue,Order Support Team
2,The speaker arrived broken right out of the bo...,Return Issue,Returns Team
3,I ordered 3 t-shirts but only received 2. The ...,Order Issue,Order Support Team
4,You sent the wrong size. I ordered L but recei...,Order Issue,Order Support Team


bert-base-uncased is a base model, NOT a classifier.
It has:
No training on your categories
No ability to classify "Order Issue / Delivery Delay / Return Issue"
No pre-defined label mapping

So it only returns junk like:
LABEL_0  
LABEL_1  
LABEL_2  
Because that’s how all untrained BERT models behave.

In [ ]:
train_df,test_df = train_test_split(df,test_size=0.2,random_state=42)

In [ ]:
test_df["zeroshot_pred"] = test_df["message_text"].apply(predict_category)

print(classification_report(
    test_df["category"],
    test_df["zeroshot_pred"]
))

                 precision    recall  f1-score   support

 Delivery Delay       0.67      0.67      0.67         3
 Fraud/Security       1.00      0.33      0.50         3
    Order Issue       0.33      1.00      0.50         1
   Return Issue       0.50      1.00      0.67         1
Technical Issue       1.00      0.67      0.80         3

       accuracy                           0.64        11
      macro avg       0.70      0.73      0.63        11
   weighted avg       0.80      0.64      0.64        11



In [ ]:
classifier = pipeline("text-classification", model="bert-base-uncased",top_k=1)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
print(classifier("message_text"))


[[{'label': 'LABEL_0', 'score': 0.5007567405700684}]]


In [ ]:
def predict_category(test):
  out = classifier(test)

  if isinstance(out[0],dict):
    return out[0]['label']

  if isinstance(out[0],list):
    top = max(out[0], key=lambda x: x['score'])
    return top['label']

  return 'unknown'

In [ ]:
test_df["predicted_category"] = test_df["message_text"].apply(predict_category)
test_df[["message_text", "category", "predicted_category"]].head()

,message_text,category,predicted_category
43,SMS received from unknown number asking to cli...,Fraud/Security,LABEL_1
40,I got a call asking for OTP for order confirma...,Fraud/Security,LABEL_0
46,Profile name suddenly changed. Has my account ...,Fraud/Security,LABEL_1
12,The parcel has been stuck at the hub for 8 day...,Delivery Delay,LABEL_0
24,I only got 1799 back out of 2999. Where is the...,Return Issue,LABEL_1


In [ ]:
routing_map = {
    "Order Issue": "Order Support Team",
    "Delivery Delay": "Logistics Team",
    "Return Issue": "Returns Team",
    "Technical Issue": "Tech Support Team",
    "Fraud/Security": "Fraud Prevention Team"
}

In [ ]:
test_df['predicted_route'] = test_df['predicted_category'].map(routing_map)

In [ ]:
test_df[['message_text','predicted_category','predicted_route']].head()

,message_text,predicted_category,predicted_route
43,SMS received from unknown number asking to cli...,LABEL_1,NaN
40,I got a call asking for OTP for order confirma...,LABEL_0,NaN
46,Profile name suddenly changed. Has my account ...,LABEL_1,NaN
12,The parcel has been stuck at the hub for 8 day...,LABEL_0,NaN
24,I only got 1799 back out of 2999. Where is the...,LABEL_1,NaN


In [ ]:
test_df["predicted_category"] = bert_model.predict(test_df["message_text"])
test_df["zeroshot_pred"] = zeroshot_model(test_df["message_text"])
